In [2]:
import os
import pandas as pd
from blockbaxAnalyser import blockbaxAnalyser

In [10]:
# --------------- SETTINGS -----------------------------------------
verbose = False
sb_numbers = [148098, 148097, 148091, 148088, 148076, 148096]
temp_steps = [-20, -10, 0, 10, 20, 30, 40, 50]
temp_steps_short = [-10, 0, 10, 20]
y_steps = [-10, -5, -2, 0, 2, 5, 10]

In [11]:
# --------------- INITIALISATION --------------------------------------
analyser = blockbaxAnalyser(sb_numbers = sb_numbers, 
plot_from_date="2022-04-28 18:30:00", 
plot_till_date="2022-05-05 01:00:00")

analyser.dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
offset_date_start="2022-05-02 03:40:00",
offset_date_end="2022-05-02 05:35:00")

if verbose:
    print(analyser.dfs)

In [16]:
bins_y = [-11, -7, -3.5, -1, 1, 3.5, 7, 10]
bins_temp = [-25, -15, -5, 5, 15, 25, 35, 45, 55]
for i in range(len(sb_numbers)):
    analyser.dfs[i]["Y setting"] = pd.cut(analyser.dfs[i]["Y value"], bins = bins_y, labels=y_steps)
    analyser.dfs[i]["Temp setting"] = pd.cut(analyser.dfs[i]["Temperature"], bins = bins_temp, labels=temp_steps)
    print(analyser.dfs[i])
    print(analyser.dfs[0].to_string())
    


                               Datetime  Temperature   Y value  Reference  \
Datetime                                                                    
2022-04-29 12:41:02 2022-04-29 12:41:02        -12.2  9.365712   9.089661   
2022-04-29 12:46:02 2022-04-29 12:46:02        -11.6  9.353505   9.091861   
2022-04-29 12:51:02 2022-04-29 12:51:02        -11.1  9.353505   9.109361   
2022-04-29 12:56:02 2022-04-29 12:56:02        -10.7  9.353505   9.098661   
2022-04-29 13:01:02 2022-04-29 13:01:02        -10.5  9.347402   9.081761   
...                                 ...          ...       ...        ...   
2022-05-02 15:15:56 2022-05-02 15:15:56         21.9  8.981191   9.174561   
2022-05-02 15:20:58 2022-05-02 15:20:58         21.9  9.091054   9.172261   
2022-05-02 15:25:58 2022-05-02 15:25:58         21.9  9.042226   9.184161   
2022-05-02 15:30:56 2022-05-02 15:30:56         21.9  9.097157   9.207361   
2022-05-02 15:35:56 2022-05-02 15:35:56         21.9  9.030019   9.200761   

[ ] Group data based on y setting and temp setting
[ ] Normalise data based on stdtransform
[ ] Plot normalised data and determine R2 and trendline
[ ] Group smartbricks based on R2 and trendline slope/intersect
[ ] Create model for well-correlated smartbricks

In [2]:
analyser.trendline()

slope: -0.017533, intercept: 1.095239
R-squared: 0.001215
slope: -0.016222, intercept: 1.060571
R-squared: 0.001008
slope: -0.002360, intercept: 0.861815
R-squared: 0.000021
slope: -0.010540, intercept: 0.969812
R-squared: 0.000431
slope: -0.010792, intercept: 1.014620
R-squared: 0.000427
slope: -0.010774, intercept: 1.035340
R-squared: 0.000409
